In [19]:
import RIIM_tools

In [20]:
from qiskit import QuantumCircuit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
import numpy as np
from qiskit.providers.aer import noise
import random
provider = IBMQ.load_account()
from qiskit.providers.aer.noise import NoiseModel

Credentials are already in use. The existing account in the session will be replaced.


In [21]:
shots=10000
error_param = 0.1

In [22]:
#Setting up a quantum circuit to test
qc = QuantumCircuit(2, 2)
qc.cx(0, 1)
qc.cx(1, 0)
qc.cx(0, 1)
qc.cx(1, 0)
qc.measure(0,0)
qc.measure(1,1)
qc.count_ops()
print(qc)

             ┌───┐     ┌───┐┌─┐   
q_0: |0>──■──┤ X ├──■──┤ X ├┤M├───
        ┌─┴─┐└─┬─┘┌─┴─┐└─┬─┘└╥┘┌─┐
q_1: |0>┤ X ├──■──┤ X ├──■───╫─┤M├
        └───┘     └───┘      ║ └╥┘
 c_0: 0 ═════════════════════╩══╬═
                                ║ 
 c_1: 0 ════════════════════════╩═
                                  


In [23]:
backend_qasm = Aer.get_backend('qasm_simulator')
provider = IBMQ.get_provider(group='open')
device = provider.get_backend('ibmq_ourense')

### Pull IBMQ device properties 
full_properties = device.properties()
coupling_map = device.configuration().coupling_map
depol_noise_model = RIIM_tools.generate_cx_error_noise_model(2, error_param)


In [24]:
### Returns un-error-mitigated observable value
RIIM_tools.q1_q2_obs(execute(qc,
                    backend_qasm,
                    shots=shots,
                    noise_model=depol_noise_model,
                    coupling_map = coupling_map,
                    optimization_level=0).result().get_counts(), shots)

0.4965

In [25]:
### Returns extrapolated observable value with RIIM
RIIM_tools.run_RIIM_extrapolation(2, qc, RIIM_tools.q1_q2_obs, backend_qasm, shots, depol_noise_model, coupling_map)

-0.003824999999999079

In [26]:
### Returns extrapolated observable value with FIIM
RIIM_tools.run_FIIM_extrapolation(2, qc, RIIM_tools.q1_q2_obs, backend_qasm, shots, depol_noise_model, coupling_map)

0.14410000000000078

In [28]:
### Returns extrapolated observable value with sampled RIIM (setting normalize_shots to True)
RIIM_tools.run_RIIM_extrapolation_sampled(2, qc, 10, True, RIIM_tools.q1_q2_obs, backend_qasm, shots, depol_noise_model, coupling_map)

0.007787499999999614